In [3]:
import torch
from torch import tensor
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

In [ ]:
class Lisflood_torch(nn.Module):
    """Representation of a reservoir in the LISFLOOD-OS hydrological model."""
    
    def __init__(self, Vc: float, Vn: float, Vn_adj: float, Vf: float, Vtot: float, Qmin: float, Qn: float, Qnd: float, At: int = 86400):
        """
        Parameters:
        -----------
        Vc: float
            Volume (m3) associated to the conservative storage
        Vn: float
            Volume (m3) associated to the normal storage
        Vn_adj: float
            Volume (m3) associated to the adjusted (calibrated) normal storage
        Vf: float
            Volume (m3) associated to the flood storage
        Vtot: float
            Total reservoir storage capacity (m3)
        Qmin: float
            Minimum outflow (m3/s)
        Qn: float
            Normal outflow (m3/s)
        Qnd: float
            Non-damaging outflow (m3/s)
        At: int
            Simulation time step in seconds.
        """
        
        # storage limits
        self.Vc = torch.tensor(Vc, dtype=torch.float64)
        self.Vn = torch.tensor(Vn, dtype=torch.float64)
        self.Vn_adj = torch.tensor(Vn_adj, dtype=torch.float64)
        self.Vf = torch.tensor(Vf, dtype=torch.float64)
        self.Vtot = torch.tensor(Vtot, dtype=torch.float64)
        
        # outflow limits
        self.Qmin = torch.tensor(Qmin, dtype=torch.float64)
        self.Qn = torch.tensor(Qn, dtype=torch.float64)
        self.Qnd = torch.tensor(Qnd, dtype=torch.float64)
        
        # time step duration in seconds
        self.At = torch.tensor(At, dtype=torch.float64)
    
    
    def timestep(self, I: float, V: float, limit_Q: bool = True, k: float = 1.2, tol: float = 1e-6) -> List[float]:
        """Given an inflow and an initial storage values, it computes the corresponding outflow
        
        Parameters:
        -----------
        I: float
            Inflow (m3/s)
        V: float
            Volume stored in the reservoir (m3)
        limit_Q: bool
            Whether to limit the outflow in the flood zone when it exceeds inflow by more than 1.2 times
        k: float
            Release coefficient. If the reservoir is in the flood zone, the outflow is limited to k times the inflow
            
        Returns:
        --------
        Q, V: List[float]
            Outflow (m3/s) and updated storage (m3)
        """
        
        # update reservoir storage with the inflow volume
        V += I * self.At
        
        # ouflow depending on the storage level
        if V < 2 * self.Vc:
            Q = np.min([self.Qmin, (V - self.Vc) / self.At])
        elif V < self.Vn:
            Q = self.Qmin + (self.Qn - self.Qmin) * (V - 2 * self.Vc) / (self.Vn - 2 * self.Vc)
        elif V < self.Vn_adj:
            Q = self.Qn
        elif V < self.Vf:
            Q = self.Qn + (self.Qnd - self.Qn) * (V - self.Vn_adj) / (self.Vf - self.Vn_adj)
            if limit_Q:
                if Q > k * I:
                    Q = np.max([k * I, self.Qn])
        elif V > self.Vf:
            # Q = np.max([(V - self.Vf - tol * self.Vtot) / self.At, np.min([self.Qnd, np.max([1.2 * I, self.Qn])])])
            Q = np.max([self.Qnd, I])
            
        # limit outflow so the final storage is between 0 and 1
        Q = np.max([np.min([Q, (V - self.Vc) / self.At]), (V - self.Vtot) / self.At])
        # Q = np.max([np.min([Q, (V - self.Vc) / self.At]), I])

        # update reservoir storage with the outflow volume
        # AV = np.min([Q * self.At, V])
        # AV = np.max([AV, V - self.Vtot])
        V -= Q * self.At
        
        assert 0 <= V, 'The volume at the end of the timestep is negative.'
        assert V <= self.Vtot, 'The volume at the end of the timestep is larger than the total reservoir capacity.'
        
        return Q, V
    
    
    def simulate(self, inflow: pd.Series, Vo: float = None, limit_Q: bool = True, routine: int = 1, k: float = 1) -> pd.DataFrame:
        """Given a inflow time series (m3/s) and an initial storage (m3), it computes the time series of outflow (m3/s) and storage (m3)
        
        Parameters:
        -----------
        inflow: pd.Series
            Time series of flow coming into the reservoir (m3/s)
        Vo: float
            Initial value of reservoir storage (m3). If not provided, it is assumed that the normal storage is the initial condition
        limit_Q: bool
            Whether to limit the outflow in the flood zone when it exceeds inflow by more than 1.2 times
        k: float
            Release coefficient. If the reservoir is in the flood zone, the outflow is limited to k times the inflow
            
        Returns:
        --------
        pd.DataFrame
            A table that concatenates the storage, inflow and outflow time series.
        """
        
        if Vo is None:
            Vo = self.Vn_adj
            
        routines = {1: self.timestep,
                    2: self.timestep2,
                    3: self.timestep3,
                    4: self.timestep4,
                    5: self.timestep5,
                    6: self.timestep6}
        
        storage = pd.Series(index=inflow.index, dtype=float, name='storage')
        outflow = pd.Series(index=inflow.index, dtype=float, name='outflow')
        # for ts in tqdm(inflow.index):
        for ts in inflow.index:
            try:
                # compute outflow and new storage
                if routine == 2:
                    Q, V = routines[routine](inflow[ts], Vo, k=k)
                elif routine == 6:
                    try:
                        Q, V = routines[routine](inflow[ts], inflow[ts - timedelta(seconds=self.At)], Vo, limit_Q=limit_Q, k=k)
                    except:
                        Q, V = routines[routine](inflow[ts], inflow[ts], Vo, limit_Q=limit_Q, k=k)
                else:
                    Q, V = routines[routine](inflow[ts], Vo, limit_Q=limit_Q, k=k)
            except:
                print(ts)
                return pd.concat((storage, inflow, outflow), axis=1)
            storage[ts] = V
            outflow[ts] = Q
            # update current storage
            Vo = V

        return pd.concat((storage, inflow, outflow), axis=1)

In [ ]:
import torch
from torch import tensor
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

class Lisflood_torch:
    """Representation of a reservoir in the LISFLOOD-OS hydrological model."""
    
    def __init__(self, Vc: float, Vn: float, Vn_adj: float, Vf: float, Vtot: float, Qmin: float, Qn: float, Qnd: float, At: int = 86400):
        # ... (same as the original code)
        # Convert all parameters to PyTorch tensors
        self.Vc = torch.tensor(Vc, dtype=torch.float64)
        self.Vn = torch.tensor(Vn, dtype=torch.float64)
        # ... (similar conversion for other parameters)

    def timestep(self, I: float, V: float, limit_Q: bool = True, k: float = 1.2, tol: float = 1e-6) -> List[float]:
        # ... (same as the original code, but using torch operations instead of numpy)
        
        V = V + I * self.At
        # ouflow depending on the storage level
        if V < 2 * self.Vc:
            Q = torch.min(self.Qmin, (V - self.Vc) / self.At)
        # ... (similar conversion for other parts of the function)

        return Q, V

    # ... (similar conversion for other methods)